In [1]:
import pandas as pd

fda_frame = pd.read_excel("../dailymed_ingest/labels_to_diseases/diseases_to_IDs/indication-list-fda-v1.xlsx")
ema_frame = pd.read_excel("../ema_indications_ingest/diseases_to_IDs/indication-list-ema-v1.xlsx")
pmda_frame = pd.read_excel("../pmda_indications_ingest/diseases_to_IDs/indication-list-pmda-v1.xlsx")

print(len(fda_frame), " items in fda frame")
print(len(ema_frame), " items in ema frame")
print(len(pmda_frame), " items in pmda frame")

212309  items in fda frame
4033  items in ema frame
5227  items in pmda frame


In [2]:
df1 = pd.merge(ema_frame, pmda_frame, how="outer")
df2 = pd.merge(df1, fda_frame, how="outer")
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]

In [3]:
print(df2)

          disease IDs                    disease ID labels  \
0       MONDO:0005101                   ulcerative colitis   
1       UMLS:C0151995      Degeneration of uterine fibroid   
2       MONDO:0005133                        endometriosis   
3       MONDO:0100244  paroxysmal nocturnal hemoglobinuria   
4       MONDO:0005417             wet macular degeneration   
...               ...                                  ...   
221564  MONDO:0005301                   multiple sclerosis   
221565  MONDO:0043797                   spinal cord injury   
221566  MONDO:0002545                 spinal cord disorder   
221567  MONDO:0021187                       hyperlipidemia   
221568  MONDO:0005439        familial hypercholesterolemia   

                                         list of diseases  \
0             ULCERATIVE COLITIS (EXCLUDING SEVERE CASES)   
1                                        UTERINE FIBROIDS   
2                                           ENDOMETRIOSIS   
3          

In [4]:
df2.to_csv("indicationList.tsv", sep="\t")
df2['drug|disease'] = df2['disease IDs'].astype(str) + '|' + df2['drug ID']
print (df2)

          disease IDs                    disease ID labels  \
0       MONDO:0005101                   ulcerative colitis   
1       UMLS:C0151995      Degeneration of uterine fibroid   
2       MONDO:0005133                        endometriosis   
3       MONDO:0100244  paroxysmal nocturnal hemoglobinuria   
4       MONDO:0005417             wet macular degeneration   
...               ...                                  ...   
221564  MONDO:0005301                   multiple sclerosis   
221565  MONDO:0043797                   spinal cord injury   
221566  MONDO:0002545                 spinal cord disorder   
221567  MONDO:0021187                       hyperlipidemia   
221568  MONDO:0005439        familial hypercholesterolemia   

                                         list of diseases  \
0             ULCERATIVE COLITIS (EXCLUDING SEVERE CASES)   
1                                        UTERINE FIBROIDS   
2                                           ENDOMETRIOSIS   
3          

In [5]:
def join_strings(series, delimiter=", "):
    return delimiter.join(series)
agg_functions = {'disease ID labels': 'first', 'drug ID Label': 'first', 'drug ID': 'first', 'disease IDs': 'first', 'active ingredients in therapy':lambda x: list(x), 'list of diseases': lambda x: list(x)}

df3 = df2.groupby(df2['drug|disease']).aggregate(agg_functions)


print(len(set(df2['disease IDs'])), " unique diseases indicated for")
print(len(set(df2['drug ID'])), " unique drugs accounted for")
print(len(set(df2['drug|disease'])), " unique drug->treats->disease edges")
    

3728  unique diseases indicated for
2353  unique drugs accounted for
15744  unique drug->treats->disease edges


In [6]:
print(df3)

                                                   disease ID labels  \
drug|disease                                                           
DOID:10017|CHEBI:7915            multiple endocrine neoplasia type 1   
DOID:12177|CHEBI:4911               common variable immunodeficiency   
DOID:12177|DRUGBANK:DB00028         common variable immunodeficiency   
DOID:14275|CHEBI:16469                                atrophic vulva   
DOID:14275|CHEBI:28689                                atrophic vulva   
...                                                              ...   
UMLS:C5848083|CHEBI:8382                         Postpartum mastitis   
UMLS:C5848632|CHEBI:17026                    Progesterone deficiency   
UMLS:C5849339|UNII:P188ANX8CK                 HER2 low breast cancer   
UMLS:C5879660|CHEBI:31632      Postpartum acute deep vein thrombosis   
UMLS:C5880575|CHEBI:36560         Low cumulative sun damage melanoma   

                                        drug ID Label          

In [7]:
for i in range(len(df3)):
    if isinstance(df3.iloc[i,4][0], float) == False:
        remove_brackets = [l.replace("{","").replace("}","").replace("'",'') for l in df3.iloc[i,4]]
        unique_list = sorted(list(set(remove_brackets)))
        df3.iloc[i,4] = unique_list
    
    unique_disease_list = sorted(list(set(df3.iloc[i,5])))
    df3.iloc[i,5] = unique_disease_list
    

In [8]:
print(df3)

                                                   disease ID labels  \
drug|disease                                                           
DOID:10017|CHEBI:7915            multiple endocrine neoplasia type 1   
DOID:12177|CHEBI:4911               common variable immunodeficiency   
DOID:12177|DRUGBANK:DB00028         common variable immunodeficiency   
DOID:14275|CHEBI:16469                                atrophic vulva   
DOID:14275|CHEBI:28689                                atrophic vulva   
...                                                              ...   
UMLS:C5848083|CHEBI:8382                         Postpartum mastitis   
UMLS:C5848632|CHEBI:17026                    Progesterone deficiency   
UMLS:C5849339|UNII:P188ANX8CK                 HER2 low breast cancer   
UMLS:C5879660|CHEBI:31632      Postpartum acute deep vein thrombosis   
UMLS:C5880575|CHEBI:36560         Low cumulative sun damage melanoma   

                                        drug ID Label          

In [9]:
df3.to_csv("indicationList.tsv", sep="\t")